In [2]:
#Copyright 2020 Vraj Shah, Arun Kumar
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

import tensorflow_data_validation as tfdv
import pandas as pd
import pandas as pd
from pandas.api.types import is_numeric_dtype
from collections import Counter,defaultdict

from tensorflow_data_validation.utils.schema_util import get_categorical_features
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

2023-10-12 11:47:04.781646: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-12 11:47:04.803672: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-12 11:47:04.803702: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-12 11:47:04.803719: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-12 11:47:04.808214: I tensorflow/core/platform/cpu_feature_g

In [3]:
testdf = pd.read_csv('../../Benchmark-Labeled-Data/data_test.csv')
test_metadata = pd.read_csv('../../Benchmark-Labeled-Data/Metadata/meta_data.csv')

print(len(testdf),len(test_metadata))
test_merged = pd.merge(testdf,test_metadata,on='Record_id')
print(len(test_merged))

print(test_merged)
y_true = test_merged.y_act.values.tolist()
print(y_true)
print(len(y_true))

1985 4233
1985
      Record_id                                  Attribute_name  \
0           285                     Net Income-Cont. Operations   
1           285                Earnings Before Interest and Tax   
2           285                                   After Tax ROE   
3           285                                        For Year   
4           285  Equity Earnings/Loss Unconsolidated Subsidiary   
...         ...                                             ...   
1980        328                                            item   
1981        343                                     load_weight   
1982       1970                                    SpecialNotes   
1983       2050                                         Website   
1984        222                                             NaN   

                  y_act  total_vals  num_nans     %_nans  num_of_dist_val  \
0               numeric        1781         0   0.000000             1693   
1               numeric   

In [4]:
cntExceptions = 0
y_tfdv = [0]*1985
prv_csv_name,csv_name = '',''
exception_indices = []

for index, row in test_merged.iterrows():       
    if index%100==0: print(index)
    col = row['Attribute_name']
    prv_csv_name = csv_name
    csv_name = '../../RawCSV/' + row['name']
    
    print(csv_name)
    print(col)
    
    if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')

    try:
        df_col = df[[col]]
    except KeyError:
        print('Exception Index:')
        print(index)
        y_tfdv[index]=1
        exception_indices.append(row)
        cntExceptions += 1
        continue
        
    st_option = tfdv.StatsOptions(enable_semantic_domain_stats=True)
    
    try:
        stats = tfdv.generate_statistics_from_dataframe(df_col,stats_options=st_option)
    except:
        print('Exception Index:')        
        print(index)
        y_tfdv[index]=1
        exception_indices.append(index)        
        continue
        
    schema = tfdv.infer_schema(statistics=stats)
    
    categ_lst = get_categorical_features(schema)
    for x in categ_lst: 
        print('Categorical')
        y_tfdv[index]=1
        break
    
    xc = schema.feature
    for x in xc:
        cnt_NLD = str(x).count('natural_language_domain')
        cnt_TD = str(x).count('time_domain')

        if cnt_NLD:
            print('Sentence')
            y_tfdv[index] = 3
        if cnt_TD:
            print('Time')
            y_tfdv[index] = 2   
        
    print()

0
../../RawCSV/fundamentals.csv
Net Income-Cont. Operations

../../RawCSV/fundamentals.csv
Earnings Before Interest and Tax

../../RawCSV/fundamentals.csv
After Tax ROE

../../RawCSV/fundamentals.csv
For Year

../../RawCSV/fundamentals.csv
Equity Earnings/Loss Unconsolidated Subsidiary

../../RawCSV/fundamentals.csv
Other Investing Activities

../../RawCSV/fundamentals.csv
Deferred Liability Charges

../../RawCSV/fundamentals.csv
Short-Term Debt / Current Portion of Long-Term Debt

../../RawCSV/fundamentals.csv
Other Financing Activities

../../RawCSV/fundamentals.csv
Net Income Adjustments

../../RawCSV/fundamentals.csv
Fixed Assets

../../RawCSV/fundamentals.csv
Total Liabilities

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
expense_OtherInvestments

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
expense_DeathBenefits

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
expense_AdminExpenses

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
expense_RealEstate

../../RawCSV/PPD_PlanLev

/tmp/ipykernel_262478/4109185584.py:15: DtypeWarning: Columns (41,142) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')



../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
income_OtherAdditions

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
UAALAmortPeriod_GASB

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
ServiceRetAge_avg

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
ServiceRetireeAge_avg

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
beneficiaries_other

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
ContributionFY

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
FairValueChange_SecLend

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
PVFB_InactiveNonVested

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
actives_tot

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
NormCostAmount_EE

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
benefits_ServiceRetirees

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
GainLoss

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
PVFNC_EE

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
payroll

../../RawCSV/PPD_PlanLevel_2017_12_11_UTF-8.csv
PlanName
Categorica

/tmp/ipykernel_262478/4109185584.py:15: DtypeWarning: Columns (40,141) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')


Categorical
Sentence

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
ActCostMeth_note

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
COLA_Verabatim

../../RawCSV/PPD_PlanLevel_2017_12_11.csv
RequiredContribution_est

../../RawCSV/KIVA.DHSv2.csv
ITN_Coverage_2005

../../RawCSV/KIVA.DHSv2.csv
ITN_Coverage_2015

../../RawCSV/KIVA.DHSv2.csv
nn.dists

../../RawCSV/KIVA.DHSv2.csv
Slope

100
../../RawCSV/KIVA.DHSv2.csv
BUILT_Population_2000

../../RawCSV/KIVA.DHSv2.csv
Unnamed: 0

../../RawCSV/KIVA.DHSv2.csv
MPI.sd

../../RawCSV/KIVA.DHSv2.csv
DHS.lat

../../RawCSV/KIVA.DHSv2.csv
MPI.median

../../RawCSV/KIVA.DHSv2.csv
Temperature_February

../../RawCSV/KIVA.DHSv2.csv
Temperature_June

../../RawCSV/KIVA.DHSv2.csv
Rainfall_1985

../../RawCSV/KIVA.DHSv2.csv
Rainfall_1990

../../RawCSV/KIVA.DHSv2.csv
Rainfall_2010

../../RawCSV/KIVA.DHSv2.csv
Enhanced_Vegetation_Index_2005

../../RawCSV/KIVA.DHSv2.csv
Aridity

../../RawCSV/KIVA.DHSv2.csv
Proximity_to_Protected_Areas

../../RawCSV/KIVA.DHSv2.csv
Proximity

...

Categorical
Sentence

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q17HirChaOther
Categorical

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q21CoreCompDatabaseDesign
Categorical

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q21CoreCompSysDesign
Categorical

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q22LangProfHaskell
Categorical

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q22LangProfLua
Categorical

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q22LangProfOther
Categorical

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q22LangProfPascal
Categorical

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q22LangProfPHP
Categorical

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q22LangProfRuby
Categorical

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q23FrameMeteor
Categorical

../../RawCSV/HackerRank-Developer-Survey-2018-Values.csv
q25LangC
Categorical

../../RawCSV/HackerRan

/tmp/ipykernel_262478/4109185584.py:15: DtypeWarning: Columns (7,8,18,25) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')


Categorical

../../RawCSV/Summary of Weather.csv
MinTemp

../../RawCSV/Summary of Weather.csv
DR

../../RawCSV/Summary of Weather.csv
MAX

../../RawCSV/Summary of Weather.csv
RVG

../../RawCSV/Summary of Weather.csv
SD3

../../RawCSV/jena_climate_2009_2016.csv
VPdef (mbar)

../../RawCSV/jena_climate_2009_2016.csv
rh (%)

../../RawCSV/Lamb History.csv
0.30%
Exception Index:
1061
../../RawCSV/2014_referrals_all.csv
2013 - 2014 % Change
Categorical

../../RawCSV/crime_incident_data2012.csv
Report Time
Categorical
Time

../../RawCSV/crime_incident_data2012.csv
Police District
Categorical

../../RawCSV/2017.csv
Trust..Government.Corruption.

../../RawCSV/2017.csv
Economy..GDP.per.Capita.

../../RawCSV/crime_incident_data2013.csv
Report Time
Categorical
Time

../../RawCSV/crime_incident_data2013.csv
Major Offense Type
Categorical

../../RawCSV/crime_incident_data2013.csv
Police Precinct
Categorical
Sentence

../../RawCSV/pokemon-added.csv
against_bug

../../RawCSV/pokemon-added.csv
against_n

/tmp/ipykernel_262478/4109185584.py:15: DtypeWarning: Columns (7,10,18,19,20,21,23,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  if prv_csv_name != csv_name:  df = pd.read_csv(csv_name,encoding='latin1')



1100
../../RawCSV/U.S._Chronic_Disease_Indicators.csv
LowConfidenceLimit

../../RawCSV/U.S._Chronic_Disease_Indicators.csv
DataValue
Exception Index:
1101
../../RawCSV/U.S._Chronic_Disease_Indicators.csv
YearEnd

../../RawCSV/U.S._Chronic_Disease_Indicators.csv
DataValueType
Categorical

../../RawCSV/U.S._Chronic_Disease_Indicators.csv
LocationAbbr
Categorical

../../RawCSV/U.S._Chronic_Disease_Indicators.csv
StratificationCategoryID1
Exception Index:
1105
../../RawCSV/U.S._Chronic_Disease_Indicators.csv
Stratification3
Categorical

../../RawCSV/U.S._Chronic_Disease_Indicators.csv
StratificationCategory3
Categorical

../../RawCSV/U.S._Chronic_Disease_Indicators.csv
StratificationID3
Categorical

../../RawCSV/eo1.csv
ORGANIZATION

../../RawCSV/eo1.csv
GROUP

../../RawCSV/tweets.csv
retweets

../../RawCSV/official_country_data_table_2_2_value_added_by_industries_at_constant_prices_isic_rev_3_data.csv
value_footnotes
Categorical

../../RawCSV/official_country_data_table_2_2_value_added_b

In [ ]:
dict_label_true = {
    'numeric': 0,
    'categorical': 1,
    'datetime': 2,
    'sentence': 3,
    'url': 4,
    'embedded-number': 5,
    'list': 6,
    'not-generalizable': 7,
    'context-specific': 8
}

y_true = [dict_label_true[str(i)] for i in y_true]

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score


def PrintMetrics(y_true, y_pred):
    print(f'Accuracy: {accuracy_score(y_true, y_pred)}')

    matrix = confusion_matrix(y_true, y_pred)
    print(matrix)

    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)

    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 score: {f1}')


PrintMetrics(y_true, y_tfdv)

NameError: name 'y_true' is not defined